### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [16]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function parabola(x)
    a = 0.13625841750841752
    b = 0.189709595959596
    c = 0.22
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [17]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [27]:
startI = -1.705
endI = -1.75
lengthI = 200

rangeI = range( startI, endI, length = lengthI )

step_to_left = rangeI.step.hi

-0.00022613065326632098

In [28]:
file = "2d_map_parabola_zoom_reverse.txt"

"2d_map_parabola_zoom_reverse.txt"

In [37]:
for (index_I0, I0) in enumerate(rangeI)
    
        if index_I0 == 1
            global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
        end
        
        U0 = parabola(I0)
    
        print_output(index_I0, I0, U0, initial_condition_function)
    
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    
        ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_function = tr[end, :]
    
        println("last point: $initial_condition_function");flush(stdout)
    
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 4, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
    
        println("----------------------")
    
        if I0 != -1.75
            move_to_left( I0, U0, initial_condition_function, step_to_left )
        end
        
end

index: 1
I0: -1.705
U0: 0.2926517650462963
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [3.07618504041362, 0.48970947753123895, 0.7789842956814212, 0.06464758424385432]
LSE: [-0.004207965412879482, -0.5555557904803415, -0.8523111511813979, -7.832413345689426]
----------------------

<<<<<<<<<<<<<<<<<<<<<
MOVE TO LEFT
 <- 
index: 1
I0: -1.7052261306532663
U0: 0.2926517650462963
initial condition: [3.07618504041362, 0.48970947753123895, 0.7789842956814212, 0.06464758424385432]
last point: [1.6873002023251409, 0.840526783468722, 0.67799179899348, 0.0657879427671468]
LSE: [0.008138906496881907, -0.5555555175634755, -0.8392423689628302, -7.840274103915949]
_________________

 <- 
index: 2
I0: -1.7054522613065326
U0: 0.2926517650462963
initial condition: [1.6873002023251409, 0.840526783468722, 0.67799179899348, 0.0657879427671468]
last point: [1.299878915807506, 0.8225566751658245, 0.6953539902419008, 0.06222909069044837]
LSE: [0.006524111227003303, -0.5555554871013596, -0.83407961305

LSE: [-0.00199194387726702, -0.5555557990424712, -0.6235996585595621, -7.8215095726073995]
_________________

 <- 
index: 23
I0: -1.7102010050251253
U0: 0.2926517650462963
initial condition: [24.307717687188763, 0.4607311125450563, 0.7363618127469667, 0.0765155814266586]
last point: [1.8007412843198873, 0.8458125601422517, 0.6689655509812497, 0.07407678312906495]
LSE: [0.005165159747376775, -0.5555558631019787, -0.6102662438660024, -7.818003530842579]
_________________

 <- 
index: 24
I0: -1.7104271356783918
U0: 0.2926517650462963
initial condition: [1.8007412843198873, 0.8458125601422517, 0.6689655509812497, 0.07407678312906495]
last point: [1.0615782190588634, 0.7850788766733944, 0.7120358752539971, 0.06723354337431096]
LSE: [0.0013127771790852004, -0.5555560038104016, -0.5954039518635055, -7.816273384819486]
_________________

 <- 
index: 25
I0: -1.710653266331658
U0: 0.2926517650462963
initial condition: [1.0615782190588634, 0.7850788766733944, 0.7120358752539971, 0.067233543374310

U0: 0.2926517650462963
initial condition: [1.3291566585854477, 0.6071463995829846, 0.7521676087639078, 0.084261287053484]
last point: [1.259050683975651, 0.8319277600031207, 0.6851261978777319, 0.08370724661279205]
LSE: [0.007023588529507679, -0.01023169756659456, -2.0553509437081763, -11.640024250576968]
_________________

 <- 
index: 46
I0: -1.7154020100502507
U0: 0.2926517650462963
initial condition: [1.259050683975651, 0.8319277600031207, 0.6851261978777319, 0.08370724661279205]
last point: [1.4450222836286388, 0.8944426570448517, 0.6233573966208794, 0.4982792430282382]
LSE: [-0.0026059545324560104, -0.03554608727607715, -2.3055422934908885, -12.341257382358386]
_________________

 <- 
index: 47
I0: -1.715628140703517
U0: 0.2926517650462963
initial condition: [1.4450222836286388, 0.8944426570448517, 0.6233573966208794, 0.4982792430282382]
last point: [1.9617829331977297, 0.8457851841365635, 0.6657809586420061, 0.3543110038034364]
LSE: [0.00981444304082429, -0.007454073517852348, -2

LSE: [-0.31243149409998966, -0.3058838443354213, -11.368086161980077, -15.56994464777529]
_________________

 <- 
index: 68
I0: -1.72037688442211
U0: 0.2926517650462963
initial condition: [0.9531876671703925, 0.9246684709501136, 0.5697981824304634, 0.48895863936599854]
last point: [0.953030485185392, 0.9246737527205928, 0.5698489461963128, 0.488978220148458]
LSE: [-0.309328192677197, -0.3091750712872849, -11.366600902178831, -15.572562837888773]
_________________

 <- 
index: 69
I0: -1.7206030150753762
U0: 0.2926517650462963
initial condition: [0.953030485185392, 0.9246737527205928, 0.5698489461963128, 0.488978220148458]
last point: [0.9528733257097852, 0.9246790340799098, 0.5698997171054181, 0.48899779890648654]
LSE: [-0.30927576857513, -0.30941568330320585, -11.365116690516176, -15.575179997270382]
_________________

 <- 
index: 70
I0: -1.7208291457286424
U0: 0.2926517650462963
initial condition: [0.9528733257097852, 0.9246790340799098, 0.5698997171054181, 0.48899779890648654]
last p

last point: [0.9495781349272572, 0.9247898516584501, 0.5709675479205707, 0.4894085000854834]
LSE: [-0.30993238367605674, -0.3127715907629679, -11.334182409115634, -15.629912261584787]
_________________

 <- 
index: 91
I0: -1.7255778894472353
U0: 0.2926517650462963
initial condition: [0.9495781349272572, 0.9247898516584501, 0.5709675479205707, 0.4894085000854834]
last point: [0.9494214647485129, 0.9247951245445224, 0.5710184747858348, 0.48942803643235616]
LSE: [-0.3102489273194136, -0.312649002535948, -11.33272013031524, -15.632508177769967]
_________________

 <- 
index: 92
I0: -1.7258040201005016
U0: 0.2926517650462963
initial condition: [0.9494214647485129, 0.9247951245445224, 0.5710184747858348, 0.48942803643235616]
last point: [0.9492648165441989, 0.9248003970713345, 0.5710694086856298, 0.4894475709475823]
LSE: [-0.31055143911393723, -0.31254070808642026, -11.331258799127202, -15.6351031930524]
_________________

 <- 
index: 93
I0: -1.7260301507537679
U0: 0.2926517650462963
initial

...

last point: [0.9371991989283015, 0.9251905217922706, 0.5751780218617967, 0.49089157349188756]
LSE: [-0.3219173906615389, -0.3169796902436036, -11.232941847573349, -15.808283951373848]
_________________

 <- 
index: 166
I0: -1.743894472361807
U0: 0.293025038663698
initial condition: [0.9371991989283015, 0.9251905217922706, 0.5751780218617967, 0.49089157349188756]
last point: [0.9370441539894171, 0.9251957783316683, 0.5752295016317296, 0.4909110110984238]
LSE: [-0.32406377124060365, -0.315047363327017, -11.231528204753971, -15.810842220551843]
_________________

 <- 
index: 167
I0: -1.7441206030150735
U0: 0.293025038663698
initial condition: [0.9370441539894171, 0.9251957783316683, 0.5752295016317296, 0.4909110110984238]
last point: [0.9368891293562073, 0.9252010346777403, 0.5752809880755734, 0.4909304474858671]
LSE: [-0.32214873218754814, -0.3171767165256424, -11.230115268915053, -15.813399928112844]
_________________

 <- 
index: 168
I0: -1.7443467336683398
U0: 0.293025038663698
initia

_________________

 <- 
index: 188
I0: -1.7488693467336662
U0: 0.293025038663698
initial condition: [0.9337928678422418, 0.925306124331105, 0.5763121109452239, 0.49131893148399014]
last point: [0.9336382646790995, 0.9253113771165248, 0.5763637364108174, 0.49133834410760274]
LSE: [-0.3209398520354067, -0.322946322145961, -11.200601565629611, -15.86698976569702]
_________________

 <- 
index: 189
I0: -1.7490954773869325
U0: 0.293025038663698
initial condition: [0.9336382646790995, 0.9253113771165248, 0.5763637364108174, 0.49133834410760274]
last point: [0.9334836813516798, 0.925316629756093, 0.576415368442462, 0.4913577556867684]
LSE: [-0.3201052684716584, -0.32400094429090076, -11.199203423334719, -15.869536212263561]
_________________

 <- 
index: 190
I0: -1.7493216080401988
U0: 0.293025038663698
initial condition: [0.9334836813516798, 0.925316629756093, 0.576415368442462, 0.4913577556867684]
last point: [0.9333291178388099, 0.9253218822519584, 0.5764670070352271, 0.49137716622938304]


last point: [1.465146686719279, 0.8379679850262941, 0.6829446202450753, 0.06837144292792614]
LSE: [0.0024351279325243, -0.5555556593525204, -0.6994347053993005, -7.852641440085797]
_________________

 <- 
index: 17
I0: -1.7104271356783918
U0: 0.2930872997063533
initial condition: [1.465146686719279, 0.8379679850262941, 0.6829446202450753, 0.06837144292792614]
last point: [1.0512278561506858, 0.7801345273731204, 0.7150422024839723, 0.06511191110656539]
LSE: [0.0018279820915795676, -0.5555557158600185, -0.68282102681264, -7.8521196603009304]
_________________

 <- 
index: 18
I0: -1.710653266331658
U0: 0.2930872997063533
initial condition: [1.0512278561506858, 0.7801345273731204, 0.7150422024839723, 0.06511191110656539]
last point: [3.738976916605548, 0.8219091978446891, 0.6602018358729328, 0.07711285184698241]
LSE: [0.007614935225125026, -0.5555555509399183, -0.6713674823851126, -7.849533725862618]
_________________

 <- 
index: 19
I0: -1.7108793969849243
U0: 0.2930872997063533
initial c

_________________

 <- 
index: 39
I0: -1.715402010050251
U0: 0.2930872997063533
initial condition: [12.477475931559269, 0.7148131567758824, 0.6683281402807484, 0.08847832821760948]
last point: [3.3662968275625267, 0.8284455421456994, 0.6500159148552165, 0.08971392344119693]
LSE: [0.002631430547821955, -0.23438439607185943, -0.5539335730578274, -7.684107438594035]
_________________

 <- 
index: 40
I0: -1.7156281407035172
U0: 0.2930872997063533
initial condition: [3.3662968275625267, 0.8284455421456994, 0.6500159148552165, 0.08971392344119693]
last point: [1.7578716470351083, 0.8514171575631773, 0.6612053268297026, 0.08527019318231305]
LSE: [0.003669732429434342, -0.18659784539202554, -0.5556544634073542, -7.647470640135062]
_________________

 <- 
index: 41
I0: -1.7158542713567835
U0: 0.2930872997063533
initial condition: [1.7578716470351083, 0.8514171575631773, 0.6612053268297026, 0.08527019318231305]
last point: [1.682992775091406, 0.8520282980720885, 0.6621340019874092, 0.08631813145

Excessive output truncated after 524308 bytes.

In [36]:
function move_to_left( I0, U0, initial_condition_function, step_to_left )
    
    println("");flush(stdout)
    println("<<<<<<<<<<<<<<<<<<<<<");flush(stdout)
    
    println("MOVE TO LEFT")
        
    range_to_left_border = range( I0 + step_to_left, endI, step = step_to_left )
        
    for (index_left, I0_left) in enumerate( range_to_left_border )
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
        end
            
        println(" <- ")
        print_output(index_left, I0_left, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0_left $U0 $(λλ[1])")
        end
        separate()
    end
end

move_to_left (generic function with 1 method)